# Push Looker Descriptions

This notebook provides an example for pushing dashboard descriptions from Secoda to Looker. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [ ]:
%pip install google google-api-core google-api-python-client google-auth google-auth-httplib2 googleapis-common-protos httplib2

In [11]:

import requests
import httplib2
import google_auth_httplib2
from google.oauth2 import service_account
from googleapiclient.discovery import build

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"
LOOKER_API_URL = "https://secoda.cloud.looker.com/api/4.0"
LOOKER_CLIENT_ID = ""
LOOKER_SECRET = ""

looker_token = None

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))

def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

def looker_credentials():
    return dict(
        headers={
            "Authorization": f"token {looker_token}",
        }
    )

def looker_auth():
    response = requests.post(
        f"{LOOKER_API_URL}/login",
        data={
            "client_id": LOOKER_CLIENT_ID,
            "client_secret": LOOKER_SECRET,
        },
    )

    if response.ok:
        looker_token = response.json().get("access_token")
        return True
    raise ConnectionError(response.text)

## Push Dashboard Descriptions

You can obtain a dashboards's descriptions with the /dashboard/dashboards API endpoint. The API endpoint returns a list of dashboards. Each dashboard has a `description` field. You can use this field to update the dashboard description in Looker.

In [20]:
def get_dashboards():
    res = session.get(
        build_url(f"/dashboard/dashboards/?integration=22208f1e-47a0-4f18-828d-18aa8fbaf34f")
    )
    results = res.json().get("results")
    if len(results) >= 1:
        return results
    return None

def patch_looker_dashboard(dashboard_id, properties):
    try:
        res = requests.patch(
            f"{LOOKER_API_URL}/dashboards/{dashboard_id}", 
            properties, 
            **looker_credentials()
        )
        print(res.text)
        print(f"Successfully updated dashboard {dashboard.get('title')} with {properties}")
    except Exception as e:
        print(e)
        print(f"Failed to update dashboard {dashboard.get('title')} with {properties}")

looker_auth()        
dashboards = get_dashboards()
for dashboard in dashboards:
    print(f"Updating dashboard {dashboard.get('title')}...")
    # Push descriptions
    if dashboard.get('description', ''):
        
        dashboard_id = dashboard.get("databuilder_id", ".").split(".")[-1]
        patch_looker_dashboard(dashboard_id, {"description": dashboard.get("description", "")})




Updating dashboard Dashboard with Multiple Explores...
Updating dashboard Orders Dashboard...
{"message":"Requires authentication.","documentation_url":"https://cloud.google.com/looker/docs/"}
Successfully updated dashboard Orders Dashboard with {'description': 'Test description...'}
Updating dashboard Orders Dashboard 2...
